ta koda za majhne mu in manjhne grafe generira vse možne grafe, najde tistega z najmanjšim subpath number in ga shrani v csv.  
ima izholjšano porabu CPU-ja
ne piše sproti rezultate v csv
REZULTATE SHRANJUJE V Za_male_grafe.csv

za mu do 4 in V do 10

In [1]:
import itertools
from sage.all import *
import pandas as pd
import os
from multiprocessing import Pool, cpu_count

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total

# -------------------------------------------------
#  Funkcija za 1 (mu, n)
# -------------------------------------------------
def procesiraj(param):
    mu_target, n = param

    kandidati = []

    for G in graphs(n):
        if not G.is_connected():
            continue
        if cyclomatic_number(G) != mu_target:
            continue

        pn = subpath_number(G)
        kandidati.append((pn, G))

    if len(kandidati) == 0:
        return None

    kandidati.sort(key=lambda x: x[0])
    min_pn, G_min = kandidati[0]
    max_pn, G_max = kandidati[-1]

    min_slika = f"slike_min/graf_min_mu{mu_target}_n{n}.png"
    max_slika = f"slike_max/graf_max_mu{mu_target}_n{n}.png"

    G_min.plot().save(min_slika)
    G_max.plot().save(max_slika)

    g6_min = G_min.graph6_string()
    g6_max = G_max.graph6_string()

    # ----------------------------------------------
    #  SPROTI PIŠEMO V LASTEN CSV
    # ----------------------------------------------
    tmp_name = f"tmp_mu{mu_target}_n{n}.csv"

    df = pd.DataFrame([
        [n, mu_target, min_pn, max_pn, min_slika, max_slika, g6_min, g6_max]
    ],
    columns=[
        "n", "µ(G)", "min p_n(G)", "max p_n(G)",
        "slika_min", "slika_max", "graph6_min", "graph6_max"
    ])

    df.to_csv(tmp_name, index=False)   # vsak proces ima svojo datoteko

    return tmp_name


# -------------------------------------------------
#  1) USTVARIMO NALOGE
# -------------------------------------------------
MUJEV = range(2,5)
N_RANGE = range(3, 11)

tasks = [(mu, n) for mu in MUJEV for n in N_RANGE]

# -------------------------------------------------
#  2) PARALELNO IZVAJANJE
# -------------------------------------------------
with Pool(cpu_count()) as p:
    tmp_files = p.map(procesiraj, tasks)

tmp_files = [f for f in tmp_files if f is not None]

# -------------------------------------------------
#  3) ZDRUŽI VSE TMP CSV FAJLE V EN KONČNI CSV
# -------------------------------------------------
final_name = "Za_male_grafe.csv"

dfs = [pd.read_csv(f) for f in tmp_files]
final_df = pd.concat(dfs, ignore_index=True)

final_df.to_csv(final_name, index=False)

# pobriši tmp fajle
for f in tmp_files:
    os.remove(f)

print("\n--- Končano. CSV se je generiral sproti. ---")



--- Končano. CSV se je generiral sproti. ---


ista koda, le da je za  mu=8  V = 9
shranjuje v mali_g_mu-8_V-9.csv
delovalo je 3min

In [ ]:
import itertools
from sage.all import *
import pandas as pd
import os
from multiprocessing import Pool, cpu_count

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total

# -------------------------------------------------
# 1) Funkcija, ki obdeluje 1 PAR (mu, n)
# -------------------------------------------------
def procesiraj_mu_n(param):
    mu_target, n = param

    kandidati = []

    for G in graphs(n):
        if not G.is_connected():
            continue

        mu = cyclomatic_number(G)
        if mu != mu_target:
            continue

        pn = subpath_number(G)
        kandidati.append((pn, G))

    if len(kandidati) == 0:
        return None

    kandidati.sort(key=lambda x: x[0])
    min_pn, G_min = kandidati[0]
    max_pn, G_max = kandidati[-1]

    min_slika = f"slike_min/graf_min_mu{mu_target}_n{n}.png"
    max_slika = f"slike_max/graf_max_mu{mu_target}_n{n}.png"

    G_min.plot().save(min_slika)
    G_max.plot().save(max_slika)

    g6_min = G_min.graph6_string()
    g6_max = G_max.graph6_string()

    return [
        n,
        mu_target,
        min_pn,
        max_pn,
        min_slika,
        max_slika,
        g6_min,
        g6_max
    ]

# -------------------------------------------------
# 2) PRIPRAVA PARALELNIH NALOG
# -------------------------------------------------

MUJEV = range(1, 9)
N_RANGE = range(3, 10)

tasks = []
for mu_target in MUJEV:
    for n in N_RANGE:
        tasks.append((mu_target, n))

# -------------------------------------------------
# 3) ZAŽENEMO PARALELNO OBDELAVO
# -------------------------------------------------

os.makedirs("slike_min", exist_ok=True)
os.makedirs("slike_max", exist_ok=True)

print(f"Uporabljam {cpu_count()} CPU jeder ...")

with Pool(cpu_count()) as p:
    results = p.map(procesiraj_mu_n, tasks)

# -------------------------------------------------
# 4) FILTRIRAJ prazne rezultate
# -------------------------------------------------

results = [r for r in results if r is not None]

# -------------------------------------------------
# 5) ZAPIŠI V CSV
# -------------------------------------------------

df = pd.DataFrame(
    results,
    columns=[
        "n",
        "µ(G)",
        "min p_n(G)",
        "max p_n(G)",
        "slika_min",
        "slika_max",
        "graph6_min",
        "graph6_max"
    ]
)

df.to_csv("mali_g_mu-8_V-9.csv", index=False)

print("\n--- IZRAČUN KONČAN ---")
print("Vsi CPU so bili uporabljeni.")


Uporabljam 8 CPU jeder ...

--- IZRAČUN KONČAN ---
Vsi CPU so bili uporabljeni.
